# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 14 2023 1:18PM,257362,10,9154591,Eywa Pharma Inc.,Released
1,Feb 14 2023 1:18PM,257362,10,9154682,Eywa Pharma Inc.,Released
2,Feb 14 2023 1:18PM,257362,10,9155187,Eywa Pharma Inc.,Released
3,Feb 14 2023 1:18PM,257362,10,9155195,Eywa Pharma Inc.,Released
4,Feb 14 2023 1:18PM,257362,10,9157266,Eywa Pharma Inc.,Released
5,Feb 14 2023 1:18PM,257362,10,9157267,Eywa Pharma Inc.,Released
6,Feb 14 2023 1:18PM,257362,10,9157268,Eywa Pharma Inc.,Released
7,Feb 14 2023 1:17PM,257365,16,ACG-02142023-1,ACG North America LLC,Released
8,Feb 14 2023 1:14PM,257363,19,TR-RG-02152023,"GCH Granules USA, Inc.",Released
9,Feb 14 2023 1:09PM,257360,20,9158430,"Exact-Rx, Inc.",Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
42,257360,Executing,1
43,257360,Released,1
44,257362,Released,7
45,257363,Released,1
46,257365,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257354,NaN,NaN,7.0
257360,NaN,1.0,1.0
257362,NaN,NaN,7.0
257363,NaN,NaN,1.0
257365,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257267,24.0,19.0,2.0
257274,1.0,3.0,1.0
257277,0.0,0.0,1.0
257294,0.0,0.0,1.0
257295,0.0,0.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257267,24,19,2
257274,1,3,1
257277,0,0,1
257294,0,0,1
257295,0,0,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257267,24,19,2
1,257274,1,3,1
2,257277,0,0,1
3,257294,0,0,1
4,257295,0,0,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257267,24,19,2
1,257274,1,3,1
2,257277,,,1
3,257294,,,1
4,257295,,,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 14 2023 1:18PM,257362,10,Eywa Pharma Inc.
7,Feb 14 2023 1:17PM,257365,16,ACG North America LLC
8,Feb 14 2023 1:14PM,257363,19,"GCH Granules USA, Inc."
9,Feb 14 2023 1:09PM,257360,20,"Exact-Rx, Inc."
11,Feb 14 2023 12:52PM,257344,15,"Brookfield Pharmaceuticals, LLC"
32,Feb 14 2023 12:52PM,257354,15,"Alliance Pharma, Inc."
39,Feb 14 2023 12:51PM,257335,15,"Alliance Pharma, Inc."
53,Feb 14 2023 12:14PM,257353,10,Ivaoes Animal Health
54,Feb 14 2023 12:12PM,257352,10,"Nextsource Biotechnology, LLC"
55,Feb 14 2023 12:12PM,257350,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 14 2023 1:18PM,257362,10,Eywa Pharma Inc.,,,7
1,Feb 14 2023 1:17PM,257365,16,ACG North America LLC,,,1
2,Feb 14 2023 1:14PM,257363,19,"GCH Granules USA, Inc.",,,1
3,Feb 14 2023 1:09PM,257360,20,"Exact-Rx, Inc.",,1,1
4,Feb 14 2023 12:52PM,257344,15,"Brookfield Pharmaceuticals, LLC",,5,16
5,Feb 14 2023 12:52PM,257354,15,"Alliance Pharma, Inc.",,,7
6,Feb 14 2023 12:51PM,257335,15,"Alliance Pharma, Inc.",,,14
7,Feb 14 2023 12:14PM,257353,10,Ivaoes Animal Health,,,1
8,Feb 14 2023 12:12PM,257352,10,"Nextsource Biotechnology, LLC",,,1
9,Feb 14 2023 12:12PM,257350,10,ISDIN Corporation,,,49


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 14 2023 1:18PM,257362,10,Eywa Pharma Inc.,7,,
1,Feb 14 2023 1:17PM,257365,16,ACG North America LLC,1,,
2,Feb 14 2023 1:14PM,257363,19,"GCH Granules USA, Inc.",1,,
3,Feb 14 2023 1:09PM,257360,20,"Exact-Rx, Inc.",1,1,
4,Feb 14 2023 12:52PM,257344,15,"Brookfield Pharmaceuticals, LLC",16,5,
5,Feb 14 2023 12:52PM,257354,15,"Alliance Pharma, Inc.",7,,
6,Feb 14 2023 12:51PM,257335,15,"Alliance Pharma, Inc.",14,,
7,Feb 14 2023 12:14PM,257353,10,Ivaoes Animal Health,1,,
8,Feb 14 2023 12:12PM,257352,10,"Nextsource Biotechnology, LLC",1,,
9,Feb 14 2023 12:12PM,257350,10,ISDIN Corporation,49,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 14 2023 1:18PM,257362,10,Eywa Pharma Inc.,7,,
1,Feb 14 2023 1:17PM,257365,16,ACG North America LLC,1,,
2,Feb 14 2023 1:14PM,257363,19,"GCH Granules USA, Inc.",1,,
3,Feb 14 2023 1:09PM,257360,20,"Exact-Rx, Inc.",1,1,
4,Feb 14 2023 12:52PM,257344,15,"Brookfield Pharmaceuticals, LLC",16,5,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 14 2023 1:18PM,257362,10,Eywa Pharma Inc.,7.0,NaN,NaN
1,Feb 14 2023 1:17PM,257365,16,ACG North America LLC,1.0,NaN,NaN
2,Feb 14 2023 1:14PM,257363,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
3,Feb 14 2023 1:09PM,257360,20,"Exact-Rx, Inc.",1.0,1.0,NaN
4,Feb 14 2023 12:52PM,257344,15,"Brookfield Pharmaceuticals, LLC",16.0,5.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 14 2023 1:18PM,257362,10,Eywa Pharma Inc.,7.0,0.0,0.0
1,Feb 14 2023 1:17PM,257365,16,ACG North America LLC,1.0,0.0,0.0
2,Feb 14 2023 1:14PM,257363,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
3,Feb 14 2023 1:09PM,257360,20,"Exact-Rx, Inc.",1.0,1.0,0.0
4,Feb 14 2023 12:52PM,257344,15,"Brookfield Pharmaceuticals, LLC",16.0,5.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5918641,208.0,2.0,0.0
15,1286574,40.0,27.0,25.0
16,257365,1.0,0.0,0.0
19,1543908,29.0,29.0,0.0
20,772044,6.0,3.0,0.0
21,257330,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5918641,208.0,2.0,0.0
1,15,1286574,40.0,27.0,25.0
2,16,257365,1.0,0.0,0.0
3,19,1543908,29.0,29.0,0.0
4,20,772044,6.0,3.0,0.0
5,21,257330,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,208.0,2.0,0.0
1,15,40.0,27.0,25.0
2,16,1.0,0.0,0.0
3,19,29.0,29.0,0.0
4,20,6.0,3.0,0.0
5,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,208.0
1,15,Released,40.0
2,16,Released,1.0
3,19,Released,29.0
4,20,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,0.0,25.0,0.0,0.0,0.0,0.0
Executing,2.0,27.0,0.0,29.0,3.0,1.0
Released,208.0,40.0,1.0,29.0,6.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,0.0,25.0,0.0,0.0,0.0,0.0
1,Executing,2.0,27.0,0.0,29.0,3.0,1.0
2,Released,208.0,40.0,1.0,29.0,6.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,0.0,25.0,0.0,0.0,0.0,0.0
1,Executing,2.0,27.0,0.0,29.0,3.0,1.0
2,Released,208.0,40.0,1.0,29.0,6.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()